In [905]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [906]:
#Cargar archivo csv
df= pd.read_csv("BASE.csv")
df.head()

,name,description,neighborhood_overview,host_name,host_since,host_location,host_about,host_response_time,host_is_superhost,host_neighbourhood,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,Villa Dante,"Dentro de Villa un estudio de arte con futon, ...","Santa Fe Shopping Mall, Interlomas Park and th...",Dici,2010-06-28,"Mexico City, Mexico","Master in visual arts, film photography & Mark...",Within an hour,f,NaN,...,4.751745,4.92,4.92,4.840647,4.713399,1.0,1.0,0.0,0.0,0.0
1,Condesa Haus,A new concept of hosting in mexico through a b...,NaN,Fernando,2010-08-09,"Mexico City, Mexico",Condesa Haus Rentals offers independent stud...,within an hour,f,Condesa,...,4.700000,4.87,4.78,4.980000,4.470000,9.0,4.0,2.0,0.0,0.0
2,Great space in historical San Rafael,This great apartment is located in one of the ...,Very traditional neighborhood with all service...,Maris,2010-10-19,"Mexico City, Mexico",I am a University Professor now retired after ...,within a few hours,f,San Rafael,...,4.880000,4.98,4.94,4.760000,4.790000,1.0,1.0,0.0,0.0,0.0
3,"2 bedroom apt. deco bldg, Condesa","Comfortably furnished, sunny, 2 bedroom apt., ...",NaN,Nicholas,2011-01-04,"Mexico City, Mexico","I am a journalist writing about food, (book an...",within a few hours,f,Hipódromo,...,4.760000,4.94,4.92,4.980000,4.920000,2.0,2.0,0.0,0.0,0.0
4,Beautiful light Studio Coyoacan- full equipped !,COYOACAN designer studio quiet & safe! well eq...,Coyoacan is a beautiful neighborhood famous fo...,Trisha,2010-08-24,"Mexico City, Mexico","I am a mother, documentary film maker and phot...",within a few hours,t,Coyoacán,...,4.960000,4.96,4.98,4.960000,4.920000,3.0,2.0,1.0,0.0,1.0


In [907]:
df.columns[df.isnull().any()]

Index(['neighborhood_overview', 'host_about', 'host_neighbourhood',
       'first_review', 'last_review'],
      dtype='object')

In [908]:
(df.isnull().mean() * 100).sort_values(ascending=False).head(5)



host_neighbourhood       48.600432
neighborhood_overview    47.085338
host_about               42.562782
first_review             12.776031
last_review              12.776031
dtype: float64

In [909]:
#Rellenamos nulos
df =df.fillna(method="bfill")
df =df.fillna(method="ffill")

/var/folders/9j/wm9ykq0j7ql95x8sql60c0680000gn/T/ipykernel_4189/1655763348.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="bfill")
/var/folders/9j/wm9ykq0j7ql95x8sql60c0680000gn/T/ipykernel_4189/1655763348.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="ffill")


In [910]:
dicotomicas = [col for col in df.columns if df[col].nunique() == 2]
dicotomicas

['host_is_superhost',
 'host_has_profile_pic',
 'host_identity_verified',
 'instant_bookable']

In [911]:
{col: df[col].unique() for col in dicotomicas}

{'host_is_superhost': array(['f', 't'], dtype=object),
 'host_has_profile_pic': array(['t', 'f'], dtype=object),
 'host_identity_verified': array(['t', 'f'], dtype=object),
 'instant_bookable': array(['f', 't'], dtype=object)}

In [912]:
df["has_availability"].value_counts()

has_availability
t    26401
Name: count, dtype: int64

In [913]:
{col: df[col].unique() for col in dicotomicas}

{'host_is_superhost': array(['f', 't'], dtype=object),
 'host_has_profile_pic': array(['t', 'f'], dtype=object),
 'host_identity_verified': array(['t', 'f'], dtype=object),
 'instant_bookable': array(['f', 't'], dtype=object)}

In [914]:
df.columns

Index(['name', 'description', 'neighborhood_overview', 'host_name',
       'host_since', 'host_location', 'host_about', 'host_response_time',
       'host_is_superhost', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'neighbourhood_cleansed', 'property_type',
       'room_type', 'amenities', 'has_availability', 'first_review',
       'last_review', 'instant_bookable', 'host_id', 'latitude', 'longitude',
       'host_response_rate', 'host_acceptance_rate', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'price', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       '

PRIMERAS CUATRO VARIABLES:

Son dicotómicas 

1era y 4ta se les aplicó regresión logística sin métodos de reponderación

2da y 3era se les aplicó método de oversampling

1er CASO: Host is superhost

In [915]:
df["host_is_superhost"].value_counts()

host_is_superhost
f    16185
t    10216
Name: count, dtype: int64

In [916]:
df["host_is_superhost"] = df["host_is_superhost"].map({'t': 1, 'f': 0})

In [917]:
df.corr(numeric_only=True)["host_is_superhost"].abs().sort_values(ascending=False).head(11)

    #Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['estimated_occupancy_l365d', 'number_of_reviews_ltm', 'number_of_reviews_ly']]
Var_Dep= df['host_is_superhost']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =42)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[4023  887]
 [1619 1392]]


In [918]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.7130450194966323


In [919]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.6107942079859587


In [920]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6836258048226234


In [921]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.8193482688391038


In [922]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.4623048820989704


In [923]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.5262759924385634


In [924]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=0)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.7625094768764216


In [925]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_is_superhost'])
unico

array([0, 1])

2NDO CASO: HOST HAS PROFILE PIC

In [926]:
df["host_has_profile_pic"].value_counts()

host_has_profile_pic
t    25838
f      563
Name: count, dtype: int64

In [927]:
df["host_has_profile_pic"] = df["host_has_profile_pic"].map({'t': 1, 'f': 0})
df.corr(numeric_only=True)["host_has_profile_pic"].abs().sort_values(ascending=False).head(10)

host_has_profile_pic                           1.000000
host_id                                        0.216958
number_of_reviews                              0.088765
maximum_maximum_nights                         0.084779
maximum_nights_avg_ntm                         0.081304
minimum_maximum_nights                         0.076060
calculated_host_listings_count                 0.068481
calculated_host_listings_count_entire_homes    0.065381
number_of_reviews_ly                           0.065310
host_is_superhost                              0.064521
Name: host_has_profile_pic, dtype: float64

In [928]:
df["host_has_profile_pic"] = df["host_has_profile_pic"].map({1: 't', 0: 'f'})

In [929]:
    #Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['number_of_reviews', 'maximum_nights_avg_ntm', 'calculated_host_listings_count']]
Var_Dep= df['host_has_profile_pic']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =42)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  177]
 [   0 7744]]


In [930]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo:')
print(precision)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad)


Precisión del modelo:
0.977654336573665
Precisión del modelo:
0.0
Exactitud del modelo:
0.977654336573665
Sensibilidad del modelo:
1.0
Sensibilidad del modelo:
0.0


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [931]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.0


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [932]:
#Aplicamos la técnica de sobremuestreo (oversampling)
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

#Entrenamos el modelo sobremuestreado
algoritmo_Over_sampling = LogisticRegression()
algoritmo_Over_sampling.fit(X_resampled, y_resampled)

#Realizamos una predicción
y_pred_over_sampling = algoritmo_Over_sampling.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred_over_sampling)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 134   43]
 [2858 4886]]


In [933]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_over_sampling, average="binary", pos_label='t')
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9912761209170217


In [934]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_over_sampling, average="binary", pos_label='f')
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.04478609625668449


In [935]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred_over_sampling)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6337583638429491


In [936]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred_over_sampling, average="binary", pos_label='t')
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.6309400826446281


In [937]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred_over_sampling, average="binary", pos_label='f')
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.7570621468926554


In [938]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred_over_sampling, average="binary", pos_label='t')
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.7710881401404561


In [939]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred_over_sampling, average="binary", pos_label='f')
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.08456926475228779


In [940]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_has_profile_pic'])
unico

array(['f', 't'], dtype=object)

CASO 3: HOST IDENTITY VERIFIED

In [941]:
df["host_identity_verified"].value_counts()

host_identity_verified
t    25228
f     1173
Name: count, dtype: int64

In [942]:
df["host_identity_verified"] = df["host_identity_verified"].map({'t': 1, 'f': 0})
df.corr(numeric_only=True)["host_identity_verified"].abs().sort_values(ascending=False).head(10)

host_identity_verified       1.000000
availability_eoy             0.241658
availability_365             0.238510
availability_90              0.226353
availability_60              0.204823
availability_30              0.170510
estimated_occupancy_l365d    0.138322
reviews_per_month            0.131971
number_of_reviews_ltm        0.131552
host_is_superhost            0.122625
Name: host_identity_verified, dtype: float64

In [943]:
df["host_identity_verified"] = df["host_identity_verified"].map({1: 't', 0: 'f'})

In [944]:
    #Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['availability_eoy', 'availability_365', 'availability_90']]
Var_Dep= df['host_identity_verified']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =42)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  366]
 [   0 7555]]


In [945]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo:')
print(precision)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad)


Precisión del modelo:
0.953793712915036
Precisión del modelo:
0.0
Exactitud del modelo:
0.953793712915036
Sensibilidad del modelo:
1.0
Sensibilidad del modelo:
0.0


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [946]:
#Aplicamos la técnica de sobremuestreo (oversampling)
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

#Entrenamos el modelo sobremuestreado
algoritmo_Over_sampling = LogisticRegression()
algoritmo_Over_sampling.fit(X_resampled, y_resampled)

#Realizamos una predicción
y_pred_over_sampling = algoritmo_Over_sampling.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred_over_sampling)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 252  114]
 [1851 5704]]


In [947]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_over_sampling, average="binary", pos_label="t")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9804056376761774


In [948]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_over_sampling, average="binary", pos_label="f")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.11982881597717546


In [949]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred_over_sampling)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7519252619618735


In [950]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred_over_sampling, average="binary", pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.7549966909331568


In [951]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred_over_sampling, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.6885245901639344


CASO 4: INSTANT BOOKABLE

In [952]:
df["instant_bookable"].value_counts()

instant_bookable
f    15697
t    10704
Name: count, dtype: int64

In [953]:
df["instant_bookable"] = df["instant_bookable"].map({'t': 1, 'f': 0})
df.corr(numeric_only=True)["instant_bookable"].abs().sort_values(ascending=False).head(10)

instant_bookable                               1.000000
host_acceptance_rate                           0.277075
calculated_host_listings_count                 0.276078
calculated_host_listings_count_entire_homes    0.212413
minimum_nights                                 0.206624
minimum_minimum_nights                         0.194622
availability_365                               0.181300
maximum_maximum_nights                         0.167875
host_id                                        0.166093
review_scores_communication                    0.159295
Name: instant_bookable, dtype: float64

In [954]:
df["instant_bookable"] = df["instant_bookable"].map({1: 't', 0: 'f'})

In [955]:
    #Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['host_acceptance_rate', 'calculated_host_listings_count', 'minimum_nights']]
Var_Dep= df['instant_bookable']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =42)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[3911  843]
 [1406 1761]]


In [956]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.6762672811059908


In [957]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.7355651683280046


In [958]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7160712031309178


In [959]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.5560467319229555


In [960]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.822675641564998


CASO 5: ROOM_TYPE

In [961]:
df["room_type"].value_counts()

room_type
Entire home/apt    17235
Private room        8867
Shared room          208
Hotel room            91
Name: count, dtype: int64

In [962]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['room_type'])
unico

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [963]:
#Convertimos una variable categórica a dicotómica 
df['room_type']= df['room_type'].replace(["Hotel room", "Private room", "Shared room"], "Rooms")

In [964]:
df["room_type"] = df["room_type"].map({'Entire home/apt': 1, 'Rooms': 0})
df.corr(numeric_only=True)["room_type"].abs().sort_values(ascending=False).head(10)

room_type                                       1.000000
calculated_host_listings_count_private_rooms    0.593159
accommodates                                    0.497233
price                                           0.417819
calculated_host_listings_count_entire_homes     0.388821
bedrooms                                        0.388738
beds                                            0.357622
estimated_revenue_l365d                         0.329649
reviews_per_month                               0.241554
estimated_occupancy_l365d                       0.239028
Name: room_type, dtype: float64

In [965]:
df["room_type"] = df["room_type"].map({1: 'Entire home/apt', 0: 'Rooms'})

In [966]:
    #Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['accommodates', 'price', 'bedrooms']]
Var_Dep= df['room_type']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =42)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[4350  864]
 [ 661 2046]]


In [967]:
df["room_type"].value_counts()

room_type
Entire home/apt    17235
Rooms               9166
Name: count, dtype: int64

In [968]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Entire home/apt")
print('Precisión del modelo:')
print(precision)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Rooms")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo:')
print(sensibilidad)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Rooms")
print('Sensibilidad del modelo:')
print(sensibilidad)

Precisión del modelo:
0.8680902015565756
Precisión del modelo:
0.7030927835051546
Exactitud del modelo:
0.8074738038126499
Sensibilidad del modelo:
0.8342922899884925
Sensibilidad del modelo:
0.7558182489841152


CASO 6: PRICE

In [969]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['price'])
unico

array([  66.,   68.,   92., ..., 3087., 3088., 3089.], shape=(2671,))

In [970]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['price'].max()
Min=df['price'].min()
Limites= [Min, Max]
Limites

[np.float64(66.0), np.float64(3089.0)]

In [971]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(65, 3088, 3)
intervalos

array([  65. , 1576.5, 3088. ])

In [972]:
#Creamos las categorías 
categorias= ["Precios medios", "Precios altos"]

# Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['price']=pd.cut(x= df['price'], bins=intervalos, labels= categorias )


In [973]:
df['price'].value_counts()

price
Precios medios    22017
Precios altos      4382
Name: count, dtype: int64

In [974]:
df["price"] = df["price"].map({'Precios medios': 1, 'Precios altos': 0})

In [975]:
print(df.corr(numeric_only=True).columns)

Index(['host_is_superhost', 'host_listings_count', 'host_total_listings_count',
       'host_id', 'latitude', 'longitude', 'host_response_rate',
       'host_acceptance_rate', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'availability_eoy',
       'number_of_reviews_ly', 'estimated_occupancy_l365d',
       'estimated_revenue_l365d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',
       'calculated_host_listings_count',
       'calculated_host_listings

In [976]:
# Limpieza + conversión a float
df['price'] = (
    df['price']
    .astype(str)
    .str.replace('[\$,]', '', regex=True)
    .str.strip()
    .replace('', np.nan)
    .astype(float)
)

print(df['price'].dtype)
print(df['price'].head())


float64
0    1.0
1    1.0
2    1.0
3    0.0
4    1.0
Name: price, dtype: float64


<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
/var/folders/9j/wm9ykq0j7ql95x8sql60c0680000gn/T/ipykernel_4189/241186245.py:5: SyntaxWarning: invalid escape sequence '\$'
  .str.replace('[\$,]', '', regex=True)


In [977]:
# Limpieza + conversión a float
df['price'] = (
    df['price']
    .astype(str)
    .str.replace('[\$,]', '', regex=True)
    .str.strip()
    .replace('', np.nan)
    .astype(float)
)

print(df['price'].dtype)
print(df['price'].head())


float64
0    1.0
1    1.0
2    1.0
3    0.0
4    1.0
Name: price, dtype: float64


<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
/var/folders/9j/wm9ykq0j7ql95x8sql60c0680000gn/T/ipykernel_4189/241186245.py:5: SyntaxWarning: invalid escape sequence '\$'
  .str.replace('[\$,]', '', regex=True)


In [978]:
df.corr(numeric_only=True)["price"].abs().sort_values(ascending=False).head(10)

price                                           1.000000
bathrooms                                       0.272580
accommodates                                    0.245115
bedrooms                                        0.237353
beds                                            0.212549
calculated_host_listings_count_private_rooms    0.152329
calculated_host_listings_count_entire_homes     0.140602
longitude                                       0.117182
estimated_revenue_l365d                         0.114118
host_total_listings_count                       0.112688
Name: price, dtype: float64

In [979]:
df['price'].isna().sum()  

np.int64(2)

In [980]:
df['price'] = df['price'].bfill()

In [981]:
df['price'].value_counts()

price
1.0    22019
0.0     4382
Name: count, dtype: int64

In [982]:
df['price'] = df['price'].astype(int).map({1: 'Precios medios', 0: 'Precios altos'})


In [983]:
    #Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['bathrooms', 'accommodates', 'bedrooms']]
Var_Dep= df['price']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =42)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  71 1282]
 [  65 6503]]


In [984]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Precios medios")
print('Precisión del modelo:')
print(precision)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Precios altos")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Precios medios")
print('Sensibilidad del modelo:')
print(sensibilidad)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Precios altos")
print('Sensibilidad del modelo:')
print(sensibilidad)

Precisión del modelo:
0.8353243416827232
Precisión del modelo:
0.5220588235294118
Exactitud del modelo:
0.8299457139250095
Sensibilidad del modelo:
0.9901035322777101
Sensibilidad del modelo:
0.0524759793052476


CASO 7: HOST LISTINGS COUNT

In [985]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_listings_count'])
unico

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  49.,  50.,  51.,  52.,  55.,  56.,  57.,  58.,
        59.,  60.,  62.,  65.,  66.,  68.,  73.,  76.,  78.,  82.,  83.,
        90.,  91.,  97.,  98., 102., 103., 104., 113., 131., 135., 138.,
       161., 178., 236., 246., 250., 258., 284., 398., 417., 442., 513.,
       896.])

In [986]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['host_listings_count'].max()
Min=df['host_listings_count'].min()
Limites= [Min, Max]
Limites

[np.float64(0.0), np.float64(896.0)]

In [987]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(-1, 897, 3)
intervalos



array([ -1., 448., 897.])

In [988]:
#Creamos las categorías 
categorias= ["Pocas propiedades", "Muchas propiedades"]

# Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['host_listings_count']=pd.cut(x= df['host_listings_count'], bins=intervalos, labels= categorias )


In [989]:
df['host_listings_count'].value_counts()

host_listings_count
Pocas propiedades     26258
Muchas propiedades      143
Name: count, dtype: int64

In [990]:
    #Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['host_total_listings_count', 'estimated_revenue_l365d', 'minimum_nights_avg_ntm']]
Var_Dep= df['host_listings_count']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =42)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  45    1]
 [   2 7873]]


In [991]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Pocas propiedades")
print('Precisión del modelo:')
print(precision)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Muchas propiedades")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Pocas propiedades")
print('Sensibilidad del modelo:')
print(sensibilidad)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Muchas propiedades")
print('Sensibilidad del modelo:')
print(sensibilidad)

Precisión del modelo:
0.9998729997459995
Precisión del modelo:
0.9574468085106383
Exactitud del modelo:
0.9996212599419265
Sensibilidad del modelo:
0.9997460317460317
Sensibilidad del modelo:
0.9782608695652174


CASO 8: HOST RESPONSE TIME

In [992]:
df["host_response_time"].value_counts()

host_response_time
within an hour        21547
within a few hours     2390
within a day           1293
a few days or more     1170
Within an hour            1
Name: count, dtype: int64

In [993]:
df['host_response_time'] = df['host_response_time'].replace({'within an hour': 'Within an hour'})

In [994]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_response_time'])
unico

array(['Within an hour', 'a few days or more', 'within a day',
       'within a few hours'], dtype=object)

In [995]:
#Convertimos una variable categórica a dicotómica 
df['host_response_time']= df['host_response_time'].replace(["a few days or more", "within a day", "within a few hours"], "More than an hour")

In [996]:
df["host_response_time"] = df["host_response_time"].map({'Within an hour': 1, 'More than an hour': 0})
df.corr(numeric_only=True)["host_response_time"].abs().sort_values(ascending=False).head(10)

host_response_time                             1.000000
host_response_rate                             0.467341
host_acceptance_rate                           0.420189
estimated_occupancy_l365d                      0.217885
reviews_per_month                              0.215385
number_of_reviews_ltm                          0.204243
host_is_superhost                              0.186501
calculated_host_listings_count_entire_homes    0.177127
number_of_reviews_ly                           0.174384
estimated_revenue_l365d                        0.165234
Name: host_response_time, dtype: float64

In [997]:
df["host_response_time"] = df["host_response_time"].map({1.0: 'Within an hour', 0.0: 'More than an hour'})

In [998]:
    #Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['host_response_rate', 'host_acceptance_rate', 'estimated_occupancy_l365d']]
Var_Dep= df['host_response_time']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =42)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 507  940]
 [ 154 6320]]


In [999]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Within an hour")
print('Precisión del modelo:')
print(precision)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="More than an hour")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Within an hour")
print('Sensibilidad del modelo:')
print(sensibilidad)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="More than an hour")
print('Sensibilidad del modelo:')
print(sensibilidad)

Precisión del modelo:
0.8705234159779615
Precisión del modelo:
0.7670196671709532
Exactitud del modelo:
0.861886125489206
Sensibilidad del modelo:
0.9762125424776027
Sensibilidad del modelo:
0.35038009675190046


CASO 9 : ACCOMODATES

In [1000]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['accommodates'].max()
Min=df['accommodates'].min()
Limites= [Min, Max]
Limites

[np.float64(1.0), np.float64(7.0)]

In [1001]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(-1, 8, 3)
intervalos

array([-1. ,  3.5,  8. ])

In [1002]:
#Creamos las categorías 
categorias= ["Pocas amenidades", "Muchas amenidades"]

# Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['accommodates']=pd.cut(x= df['accommodates'], bins=intervalos, labels= categorias )


In [1003]:
df['accommodates'].value_counts()

accommodates
Pocas amenidades     17269
Muchas amenidades     9132
Name: count, dtype: int64

In [1004]:
df['accommodates'].isna().sum()

np.int64(0)

In [1005]:
    #Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['bedrooms', 'beds', 'bathrooms']]
Var_Dep= df['accommodates']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =42)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[2004  790]
 [ 411 4716]]


In [1006]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Pocas amenidades")
print('Precisión del modelo:')
print(precision)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Muchas amenidades")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Pocas amenidades")
print('Sensibilidad del modelo:')
print(sensibilidad)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Muchas amenidades")
print('Sensibilidad del modelo:')
print(sensibilidad)

Precisión del modelo:
0.8565201598256448
Precisión del modelo:
0.8298136645962733
Exactitud del modelo:
0.8483777300845853
Sensibilidad del modelo:
0.919836161497952
Sensibilidad del modelo:
0.7172512526843235


CASO 10: MINIMUM NIGHTS

In [1007]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['minimum_nights'])
unico

array([1. , 1.6, 2. , 3. ])

In [1008]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['minimum_nights'].max()
Min=df['minimum_nights'].min()
Limites= [Min, Max]
Limites

[np.float64(1.0), np.float64(3.0)]

In [1009]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(0, 4, 3)
intervalos

array([0., 2., 4.])

In [1010]:
#Creamos las categorías 
categorias= ["0-2 NOCHES", "3-4 NOCHES"]

# Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['minimum_nights']=pd.cut(x= df['minimum_nights'], bins=intervalos, labels= categorias )


In [1011]:
df['minimum_nights'].value_counts()

minimum_nights
0-2 NOCHES    23574
3-4 NOCHES     2827
Name: count, dtype: int64

In [1012]:
    #Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['minimum_minimum_nights', 'calculated_host_listings_count', 'availability_30']]
Var_Dep= df['minimum_nights']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =42)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[7019   46]
 [ 130  726]]


In [1013]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="0-2 NOCHES")
print('Precisión del modelo:')
print(precision)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="3-4 NOCHES")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="0-2 NOCHES")
print('Sensibilidad del modelo:')
print(sensibilidad)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="3-4 NOCHES")
print('Sensibilidad del modelo:')
print(sensibilidad)

Precisión del modelo:
0.9818156385508463
Precisión del modelo:
0.9404145077720207
Exactitud del modelo:
0.9777805832596894
Sensibilidad del modelo:
0.9934890304317056
Sensibilidad del modelo:
0.8481308411214953
